In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_raw = pd.read_csv('./206_common_users.csv')

In [ ]:
df_raw.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
df_raw.head()

In [ ]:
ratings_df = df_raw.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
ratings_df.head()

In [ ]:
ratings = ratings_df.values

In [ ]:
mean_ratings = (np.true_divide(ratings.sum(1),(ratings!=0).sum(1))).reshape(-1, 1)

In [ ]:
ratings_normalized = []

In [ ]:
i = 0
for row in ratings:
    new_mat = []
#     ratings_normalized.append(row[row.nonzero()] - mean_ratings[i][0])
    for val in row:
        if val != 0:
            new_mat.append(val - mean_ratings[i][0])
        else:
            new_mat.append(0)
    ratings_normalized.append(new_mat)
    i += 1

In [ ]:
# a = np.array(a, dtype=...)
ratings_normalized = np.array(ratings_normalized, dtype='float32')

In [ ]:
from scipy.sparse.linalg import svds

In [ ]:
U, sigma, Vt = svds(ratings_normalized, k = 50)

In [ ]:
sigma = np.diag(sigma)

In [ ]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + mean_ratings.reshape(-1, 1)
# preds_df = pd.DataFrame(all_user_predicted_ratings, columns = df_raw.columns)

In [ ]:
all_user_predicted_ratings

In [ ]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = ratings_df.columns)

In [ ]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.userId == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'movieId', right_on = 'movieId'))

    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending the highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['movieId'].isin(user_full['movieId'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movieId',
               right_on = 'movieId').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [ ]:
movies_df = pd.read_csv('../206_full_data.csv')

In [ ]:
already_rated, predictions = recommend_movies(preds_df, 837, movies_df, df_raw, 10)